In [2]:
import requests
from datetime import datetime
import matplotlib.pyplot as plt

# If you already have an Etherscan API key and an Ethereum address you want to monitor, replace the API_KEY and ADDRESS variables with the correct values.
# Otherwise, you will need to register on Etherscan to obtain your API key and choose an Ethereum address you want to monitor.

API_KEY = "YourAPIKey"
ADDRESS = "YourAddress"
BASE_URL = "https://api.etherscan.io/api"
ETHER_VALUE = 10 ** 18

def make_api_url(module, action, address, **kwargs):
    url = f"{BASE_URL}?module={module}&action={action}&address={address}&apikey={API_KEY}"
    for key, value in kwargs.items():
        url += f"&{key}={value}"
    return url

def get_account_balance(address):
    balance_url = make_api_url("account", "balance", address, tag="latest", x="2")
    with requests.Session() as session:
        response = session.get(balance_url)
    data = response.json()
    value = int(data["result"]) / ETHER_VALUE
    return value

def get_transactions(address):
    transactions = []
    page = 1
    while True:
        transactions_url = make_api_url("account", "txlist", address, startblock=0, endblock=99999999, page=page, offset=10000, sort="asc")
        with requests.Session() as session:
            response = session.get(transactions_url)
        data = response.json().get("result", [])
        if not data:
            break
        transactions.extend(data)
        page += 1
    return transactions

def get_internal_transactions(address):
    internal_transactions = []
    page = 1
    while True:
        internal_tx_url = make_api_url("account", "txlistinternal", address, startblock=0, endblock=99999999, page=page, offset=10000, sort="asc")
        with requests.Session() as session:
            response = session.get(internal_tx_url)
        data = response.json().get("result", [])
        if not data:
            break
        internal_transactions.extend(data)
        page += 1
    return internal_transactions

def plot_transactions(address):
    transactions = get_transactions(address)
    internal_transactions = get_internal_transactions(address)
    transactions.extend(internal_transactions)
    transactions.sort(key=lambda x: int(x["timeStamp"]))

    current_balance = 0
    balances = []
    times = []

    for tx in transactions:
        to = tx.get("to", "")
        from_addr = tx.get("from", "")
        value = float(tx.get("value", 0)) / ETHER_VALUE
        gas = float(tx.get("gasUsed", 0)) * float(tx.get("gasPrice", 0)) / ETHER_VALUE
        time = datetime.fromtimestamp(int(tx["timeStamp"]))
        money_in = to.lower() == address.lower()
        if money_in:
            current_balance += value
        else:
            current_balance -= value + gas
        balances.append(current_balance)
        times.append(time)

    plt.plot(times, balances)
    plt.xlabel("Time")
    plt.ylabel("Balance (ETH)")
    plt.title("Account Balance Over Time")
    plt.show()

address = "YourAddress"
plot_transactions(address)
eth = get_account_balance(address)
print("Final Account Balance:", eth)

KeyboardInterrupt: 